In [ ]:
import pandas as pd
import numpy as np
from sklearn.neighbors import KNeighborsClassifier
from matplotlib import pyplot as plt
from sklearn.model_selection import learning_curve
from sklearn.datasets import load_breast_cancer
from sklearn.ensemble import RandomForestClassifier

Load and check data



In [ ]:
#Load the Data
train = pd.read_csv('../input/santander-customer-transaction-prediction/train.csv')

Below is the first 5 rows of test dataset:

In [ ]:
train.head()

The dimension and number of missing values in the train dataset is as below:

In [ ]:
print(f'Number of rows: {train.shape[0]};  Number of columns: {train.shape[1]}; No of missing values: {sum(train.isna().sum())}')

Infos

In [ ]:
train.info()

Summarie and statistics

In [ ]:
train.describe().T

In [ ]:
target_count = train['target'].value_counts().sort_index()
target_count_df = pd.DataFrame(target_count)
#pd.options.display.float_format = '{:,.2f}%'.format
target_count_df['target(%)'] = (target_count_df/target_count.sum()*100)
target_count_df.sort_values('target(%)', ascending=False, inplace=True)
display(target_count_df)

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(17, 8))

target_count = train['target'].value_counts().sort_index()

ax.bar(target_count.index, target_count, color=['#1520E6' if i%2==0 else '#93D1FF' for i in range(9)],
       width=0.55, 
       edgecolor='black', 
       linewidth=0.7)

ax.margins(0.02, 0.05)

for i in range(1,2):
    ax.annotate(f'{target_count[i]/len(train)*100:.3}', xy=(i, target_count[i]+1000),
                   va='center', ha='center',
               )
#Annotate the point xy with text text.

#In the simplest form, the text is placed at xy.

ax.set_title('target Distribution', weight='bold', fontsize=15)
ax.grid(axis='y', linestyle='-', alpha=0.4)

fig.tight_layout()
plt.show()

In [ ]:
import seaborn as sns
from matplotlib import pyplot as plt
train.isnull().sum()
sns.heatmap(train.isnull(), cbar = True).set_title("Missing values heatmap")
plt.gcf().set_size_inches(16,6)

In [ ]:
train.nunique()

In [ ]:
# Check the data
train.isnull().any().describe()

In [ ]:
train.dtypes

In [ ]:
y=train['target']
X=train.drop(labels=['target','ID_code'], axis=1)

In [ ]:
# explore lightgbm number of trees effect on performance
from numpy import mean
from numpy import std
from sklearn.datasets import make_classification
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RepeatedStratifiedKFold
from lightgbm import LGBMClassifier
from matplotlib import pyplot
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split
X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.2, random_state=42)
X_train.shape, X_valid.shape, y_train.shape, y_valid.shape

In [ ]:
import lightgbm
from sklearn.metrics import roc_auc_score
#Step2: Create a simple Light GBM Model and evaluate performance
#LightGBM has function Dataset to read the data. This is required for using LightGBM
train_data = lightgbm.Dataset(X_train, label=y_train)
valid_data = lightgbm.Dataset(X_valid, label=y_valid)
# to record eval results for plotting
evals_result = {} 
params = {
    'task': 'train',
    'boosting_type': 'gbdt',
    'objective': 'regression',
    'metric': {'l2'},
    'num_leaves': 20,
    'learning_rate': 0.05,
    'feature_fraction': 0.9,
    'bagging_fraction': 0.8,
    'bagging_freq': 5,
    'min_data_in_leaf':4,
     #'min_sum_hessian_in_leaf': 5,
    'verbose':10
}

model_lgbm = lightgbm.train(params,
                            train_data,
                            valid_sets=valid_data,
                            evals_result=evals_result,
                            num_boost_round=5000,
                            early_stopping_rounds=50)
y_train_pred = model_lgbm.predict(X_train)
y_valid_pred = model_lgbm.predict(X_valid)

print("AUC Train: {:.4f}\nAUC Valid: {:.4f}".format(roc_auc_score(y_train, y_train_pred),
                                                    roc_auc_score(y_valid, y_valid_pred)))


In [ ]:
print('Plot metrics during training...')
ax = lightgbm.plot_metric(evals_result)
plt.show()

In [ ]:
print('Plot feature importances...')
ax = lightgbm.plot_importance(model_lgbm, max_num_features=10)
plt.show()

#so  get 10 feature importance var_81 var_139 var_12 var_53 var_166 var_6 var_174 var_22 var_21 var_110

In [ ]:
def pie(df,var):
# Creating dataset
 labels = ['0', '1']


# Creating explode data
 explode = (0.1, 0.0)

# Creating color parameters
 colors = ( "orange", "cyan", "brown",
		"grey", "indigo", "beige")

# Wedge properties
 wp = { 'linewidth' : 1, 'edgecolor' : "green" }

# Creating autocpt arguments
 def func(pct, allvalues):
	 absolute = int(pct / 100.*np.sum(allvalues))
	 return "{:.1f}%\n({:d} g)".format(pct, absolute)

# Creating plot
 fig, ax = plt.subplots(figsize =(10, 7))
 wedges, texts, autotexts = ax.pie(df,
								autopct = lambda pct: func(pct, df),
								explode = explode,
								labels = labels,
								shadow = True,
								colors = colors,
								startangle = 90,
								wedgeprops = wp,
								textprops = dict(color ="magenta"))

# Adding legend
 ax.legend(wedges, labels,
		title ="target",
		loc ="center left",
		bbox_to_anchor =(1, 0, 0.5, 1))

 plt.setp(autotexts, size = 8, weight ="bold")
 ax.set_title(f"Customizing pie chart for {var}")

# show plot
 plt.show()

In [ ]:
data=train.iloc[:,[1,83]]

#data1.sort_values(by=['target'])
df=data['target'].value_counts()
pie(df,'var_81')

In [ ]:
data=train.iloc[:,[1,141]]

#data1.sort_values(by=['target'])
df=data['target'].value_counts()
pie(df,'var_139')

In [ ]:
data=train.iloc[:,[1,14]]

#data1.sort_values(by=['target'])
df=data['target'].value_counts()
pie(df,'var_12')

In [ ]:
data=train.iloc[:,[1,55]]

#data1.sort_values(by=['target'])
df=data['target'].value_counts()
pie(df,'var_53')

In [ ]:
data=train.iloc[:,[1,168]]

#data1.sort_values(by=['target'])
df=data['target'].value_counts()

pie(df,'var_166')

In [ ]:
data=train.iloc[:,[1,8]]
data.head()
#data1.sort_values(by=['target'])
df=data['target'].value_counts()
#df
pie(df,'var_6')

In [ ]:
data=train.iloc[:,[1,176]]

#data1.sort_values(by=['target'])
df=data['target'].value_counts()
pie(df,'var_174')

In [ ]:
data=train.iloc[:,[1,24]]

#data1.sort_values(by=['target'])
df=data['target'].value_counts()
pie(df,'var_22')

In [ ]:
data=train.iloc[:,[1,23]]

#data1.sort_values(by=['target'])
df=data['target'].value_counts()
pie(df,'var_21')

In [ ]:
data=train.iloc[:,[1,112]]

#data1.sort_values(by=['target'])
df=data['target'].value_counts()
pie(df,'var_110')

In [ ]:
def plt_Density(data1,data0,var):
  fig, ax = plt.subplots(1,1)


  sns.distplot(data1, color='red',
             hist_kws={"edgecolor": 'white'}, label='taeget 1')

  sns.distplot(data0, color='blue',
             hist_kws={"edgecolor": 'black'}, label='target 0')
  # visualizing plot using matplotlib.pyplot library
  # Plot formatting
  plt.legend(prop={'size': 16}, title = 'var')
  plt.title(f'Density Plot of {var}  with Multiple target 0 and target 1')
  plt.xlabel('Delay (min)')
  plt.ylabel('Density')
  plt.show()

In [ ]:
#select_price = df.loc[data['target'] == 0]

data=train.iloc[:,[1,83]]
data0 = data.loc[data['target'] == 0]
data1 = data.loc[data['target'] == 1]
plt_Density(data1.var_81,data0.var_81,'var_81')

In [ ]:
#select_price = df.loc[data['target'] == 0]

data=train.iloc[:,[1,83]]
data0 = data.loc[data['target'] == 0]
data1 = data.loc[data['target'] == 1]
plt_Density(data1.var_81,data0.var_81,'var_81')

In [ ]:
#select_price = df.loc[data['target'] == 0]

data=train.iloc[:,[1,141]]
data0 = data.loc[data['target'] == 0]
data1 = data.loc[data['target'] == 1]
plt_Density(data1.var_139,data0.var_139,'var_139')

In [ ]:
#select_price = df.loc[data['target'] == 0]

data=train.iloc[:,[1,14]]
data0 = data.loc[data['target'] == 0]
data1 = data.loc[data['target'] == 1]
plt_Density(data1.var_12,data0.var_12,'var_12')

In [ ]:
#select_price = df.loc[data['target'] == 0]

data=train.iloc[:,[1,55]]
data0 = data.loc[data['target'] == 0]
data1 = data.loc[data['target'] == 1]
plt_Density(data1.var_53,data0.var_53,'var_53')

In [ ]:
data=train.iloc[:,[1,168]]
data0 = data.loc[data['target'] == 0]
data1 = data.loc[data['target'] == 1]
plt_Density(data1.var_166,data0.var_166,'var_166')

In [ ]:
data=train.iloc[:,[1,8]]
data0 = data.loc[data['target'] == 0]
data1 = data.loc[data['target'] == 1]
plt_Density(data1.var_6,data0.var_6,'var_6')

In [ ]:
#select_price = df.loc[data['target'] == 0]

data=train.iloc[:,[1,176]]
data0 = data.loc[data['target'] == 0]
data1 = data.loc[data['target'] == 1]
plt_Density(data1.var_174,data0.var_174,'var_174')

In [ ]:
data=train.iloc[:,[1,24]]
data0 = data.loc[data['target'] == 0]
data1 = data.loc[data['target'] == 1]
plt_Density(data1.var_22,data0.var_22,'var_22')

In [ ]:
data=train.iloc[:,[1,23]]
data0 = data.loc[data['target'] == 0]
data1 = data.loc[data['target'] == 1]
plt_Density(data1.var_21,data0.var_21,'var_21')

In [ ]:
data=train.iloc[:,[1,112]]
data0 = data.loc[data['target'] == 0]
data1 = data.loc[data['target'] == 1]
plt_Density(data1.var_110,data0.var_110,'var_110')

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from matplotlib import colors
from matplotlib.ticker import PercentFormatter

def hst(data):


# Creating distribution
  n_bins = 20
  legend = ['distribution']

# Creating histogram
  fig, axs = plt.subplots(1, 1,
						figsize =(10, 7),
						tight_layout = True)


# Remove axes splines
  for s in ['top', 'bottom', 'left', 'right']:
	  axs.spines[s].set_visible(False)

# Remove x, y ticks
  axs.xaxis.set_ticks_position('none')
  axs.yaxis.set_ticks_position('none')

# Add padding between axes and labels
  axs.xaxis.set_tick_params(pad = 5)
  axs.yaxis.set_tick_params(pad = 10)

# Add x, y gridlines
  axs.grid(b = True, color ='grey',
		linestyle ='-.', linewidth = 0.5,
		alpha = 0.6)

# Add Text watermark
  fig.text(0.9, 0.15, '',
		fontsize = 12,
		color ='red',
		ha ='right',
		va ='bottom',
		alpha = 0.7)

# Creating histogram
  N, bins, patches = axs.hist(data, bins = n_bins)

# Setting color
  fracs = ((N**(1 / 5)) / N.max())
  norm = colors.Normalize(fracs.min(), fracs.max())

  for thisfrac, thispatch in zip(fracs, patches):
	  color = plt.cm.viridis(norm(thisfrac))
	  thispatch.set_facecolor(color)

# Adding extra features
  plt.xlabel("X-axis")
  plt.ylabel("y-axis")
  plt.legend(legend)
  plt.title('Customized histogram')

# Show plot
  plt.show()


In [ ]:
data=train.iloc[:,[1,83]]
hst(data.var_81)

In [ ]:
data=train.iloc[:,[1,141]]
hst(data.var_139)


In [ ]:
data=train.iloc[:,[1,14]]
hst(data.var_12)


In [ ]:
data=train.iloc[:,[1,55]]
hst(data.var_53)

In [ ]:
data=train.iloc[:,[1,168]]
hst(data.var_166)

In [ ]:
data=train.iloc[:,[1,8]]
hst(data.var_6)

In [ ]:
data=train.iloc[:,[1,176]]
hst(data.var_174)

In [ ]:
data=train.iloc[:,[1,24]]
hst(data.var_22)

In [ ]:
data=train.iloc[:,[1,23]]
hst(data.var_21)

In [ ]:
data=train.iloc[:,[1,112]]
hst(data.var_110)

In [ ]:
data=train.iloc[:,[1,8,14,23,24,55,83,112,141,168,176]]
data.head(4)

In [ ]:
import seaborn as sns

In [ ]:
sns.relplot(x='var_6', y='var_12', hue='var_21', size='var_22', col='target', data=data)

In [ ]:
sns.heatmap(data.corr(), annot=True, fmt='.2f')


In [ ]:
sns.pairplot(data)

Now let’s do the pairplot showing the charts segmented according to the values of the categorical variable

In [ ]:
sns.pairplot(data, hue='target')

In [ ]:
sns.jointplot(x='var_21', y='var_22', data=data)

Another interesting graphic is the ViolinPlot: